### This is notes for me to learn PyRTL


In [25]:
#!pip install pyrtl
#reset working space each time
import pyrtl
pyrtl.reset_working_block()

In [26]:
from IPython.display import Javascript, display
display(Javascript('''var script=document.createElement('script');script.src='https://code.jquery.com/jquery-3.6.0.min.js';document.head.appendChild(script);'''))

<IPython.core.display.Javascript object>

In PyRTL, every hardware signal — inputs, outputs, registers, wires — must be explicitly declared as a wire with a fixed bit width.
These declarations are not variables (like Python integers) — they are hardware ports that will exist on your simulated chip.

```pyrtl.Input(width, name)```
defines an input wire (like a pin on your chip).
It represents a value coming from outside the circuit each clock cycle.

Here we create four 16-bit inputs for each vector a and b.

They correspond to the elements of two 4-element vectors.

Each of these wires can carry a number between 0 and 2^16 − 1 (unsigned by default), or you can interpret them as signed fixed-point later if needed.

```pyrtl.Output(width, name)``` defines an output port — a signal that leaves the circuit.

We assign to it using <<=, which means "connect this wire to that logic."

In [27]:
#a and b are inputs; output is out lol
#the names of each wire object is just named 'a0', etc.
a = [pyrtl.Input(16, f'a{i}') for i in range(4)]
b = [pyrtl.Input(16, f'b{i}') for i in range(4)]
out = pyrtl.Output(32, 'out')

In [28]:
#parallel multiplies
products = [a[i] * b[i] for i in range(4)]

#add tree (reduction)
sum1 = products[0] + products[1]
sum2 = products[2] + products[3]
total = sum1 + sum2

out <<= total

Vocab: 
A clock cycle is the fundamental unit of time in digital hardware.
In hardware, everything runs in lockstep with a clock signal — a repeating waveform that ticks at a fixed rate. (If you play minecraft, its analogous to how things in minecraft run on ticks; except that like there are billions of cycles per second instead of 20 tps)

During a cycle:

1. Inputs change (driven by the previous register values or external signals).

2. Logic computes outputs.

3. The clock ticks → registers update.

4. The process repeats.

During simulation:
 * We provide a dictionary of input values per cycle (this is the ```inputs = ...``` section)
 * PyRTL will drive those input wires with your numbers for that clock cycle.
 * The circuit that was built (in our case, the multiplier + adder tree) computes ```out```.
 * You can inspect ```out``` using ```sim.inspect(out)```.

In [29]:
#simulation
sim_trace = pyrtl.SimulationTrace()
sim = pyrtl.Simulation(tracer=sim_trace)

# Provide example inputs and simulate a few cycles
for cycle in range(3):
    inputs = {f'a{i}': i+1 for i in range(4)}   # a = [1,2,3,4]
    inputs.update({f'b{i}': (i+2) for i in range(4)})  # b = [2,3,4,5]
    sim.step(inputs)
    print(f"Cycle {cycle}: out = {sim.inspect(out)}")

#this is what it should render as:
#if this doesnt work, save it as an html file and view it externally lmao
sim_trace.render_trace()

Cycle 0: out = 40
Cycle 1: out = 40
Cycle 2: out = 40


<IPython.core.display.Javascript object>

HOLDUP A MINUTE!
Why is my cycle 0 to cycle 2 output the same??? What does this mean?

There are no registers (flip-flops, memory, etc.), meaning:
* The computation happens instantly within one cycle (cuz we fast as fuq)
* Nothing depends on a previous tick.
* So every tick (cycle 0, 1, 2, …), we're just re-evaluating the same logic → same result.

Let's compare our parallel multiply concept with the 'default' multiply via cycle counts (speed)

In [50]:
import pyrtl
pyrtl.reset_working_block()

# Inputs/Outputs
a = [pyrtl.Input(16, f'a{i}') for i in range(4)]
b = [pyrtl.Input(16, f'b{i}') for i in range(4)]
out = pyrtl.Output(32, 'out')

# Registers
acc = pyrtl.Register(32, 'acc')
counter = pyrtl.Register(2, 'counter')

# Multiplexer to select element based on counter
current_a = pyrtl.mux(counter == 0, a[0],
               pyrtl.mux(counter == 1, a[1],
               pyrtl.mux(counter == 2, a[2], a[3])))
current_b = pyrtl.mux(counter == 0, b[0],
               pyrtl.mux(counter == 1, b[1],
               pyrtl.mux(counter == 2, b[2], b[3])))

# Multiply and accumulate
current_product = current_a * current_b
current_product.name = 'current_product'  # <-- Add this line!

acc.next <<= acc + current_product
out <<= acc

# Increment counter
counter.next <<= counter + 1

# Simulation
sim = pyrtl.Simulation()
print("Sequential multiply:")
for cycle in range(4):
    inputs = {f'a{i}': i+1 for i in range(4)}
    inputs.update({f'b{i}': i+2 for i in range(4)})
    sim.step(inputs)
    
    # Debug: inspect the mux outputs
    cnt = sim.inspect('counter')
    current_a.name = 'current_a'
    current_b.name = 'current_b'
    current_a = pyrtl.select(counter, falsecase=a[0], truecase={i: a[i] for i in range(4)})
    current_b = pyrtl.select(counter, falsecase=a[0], truecase={i: b[i] for i in range(4)})
    prod = sim.inspect('current_product')
    acc_val = sim.inspect('acc')
    
    print(f"Cycle {cycle}: counter={cnt}, current_a={curr_a}, current_b={curr_b}, product={prod}, acc={acc_val}")

Sequential multiply:


PyrtlError: error, expecting a wirevector, int, or Verilog-style const string got {0: a0/16I, 1: a1/16I, 2: a2/16I, 3: a3/16I} instead